<a href="https://colab.research.google.com/github/microsoft/autogen/blob/main/notebook/agentchat_groupchat_research.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Auto Generated Agent Chat: Performs Research with Multi-Agent Group Chat

AutoGen offers conversable agents powered by LLM, tool or human, which can be used to perform tasks collectively via automated chat. This framwork allows tool use and human participance through multi-agent conversation.
Please find documentation about this feature [here](https://microsoft.github.io/autogen/docs/Use-Cases/agent_chat).

## Requirements

AutoGen requires `Python>=3.8`. To run this notebook example, please install:
```bash
pip install pyautogen
```

In [1]:
%%capture --no-stderr
# %pip install pyautogen~=0.1.0

## Set your API Endpoint

The [`config_list_from_json`](https://microsoft.github.io/autogen/docs/reference/oai/openai_utils#config_list_from_json) function loads a list of configurations from an environment variable or a json file.

In [1]:
import autogen

config_list_gpt4 = autogen.config_list_from_json(
    "OAI_CONFIG_LIST.json",
    filter_dict={
        "model": ["JulyChat"],
    },
)

config_list_gpt4

[{'model': 'JulyChat',
  'api_key': '0a6814df9ee746eb88592a0364a02a6c',
  'api_base': 'https://zewdchat.openai.azure.com/',
  'api_type': 'azure',
  'api_version': '2023-07-01-preview'}]

It first looks for environment variable "OAI_CONFIG_LIST" which needs to be a valid json string. If that variable is not found, it then looks for a json file named "OAI_CONFIG_LIST". It filters the configs by models (you can filter by other keys as well).

The config list looks like the following:
```python
config_list = [
    {
        'model': 'gpt-4-32k',
        'api_key': '<your OpenAI API key here>',
    },
    {
        'model': 'gpt-4-32k',
        'api_key': '<your Azure OpenAI API key here>',
        'api_base': '<your Azure OpenAI API base here>',
        'api_type': 'azure',
        'api_version': '2023-06-01-preview',
    },
    {
        'model': 'gpt-4-32k-0314',
        'api_key': '<your Azure OpenAI API key here>',
        'api_base': '<your Azure OpenAI API base here>',
        'api_type': 'azure',
        'api_version': '2023-06-01-preview',
    },
]
```

If you open this notebook in colab, you can upload your files by clicking the file icon on the left panel and then choose "upload file" icon.

You can set the value of config_list in other ways you prefer, e.g., loading from a YAML file.

## Construct Agents

In [2]:
import random
gpt4_config = {
    "model": "JulyChat",
    "seed": random.randint(0, 10000),  # change the seed for different trials
    "temperature": 0.2,
    "config_list": config_list_gpt4,
    "request_timeout": 120,
}
user_proxy = autogen.UserProxyAgent(
   name="Admin",
   system_message="You are Admin, You Interact with the planner to discuss the plan and approve the plan from planner. You say TERMINATE to when task completed.",
   code_execution_config=False,
   llm_config=gpt4_config,
   human_input_mode="TERMINATE"
)
engineer = autogen.AssistantAgent(
    name="Engineer",
    llm_config=gpt4_config,
    system_message='''You are Engineer. You follow an approved plan. You write python/shell code to solve tasks. Wrap the code in a code block that specifies the script type. The user can't modify your code. So do not suggest incomplete code which requires others to modify. Don't use a code block if it's not intended to be executed by the executor.
Don't include multiple code blocks in one response. Do not ask others to copy and paste the result. Check the execution result returned by the executor.
If the result indicates there is an error, fix the error and output the code again. Suggest the full code instead of partial code or code changes. If the error can't be fixed or if the task is not solved even after the code is executed successfully, analyze the problem, revisit your assumption, collect additional info you need, and think of a different approach to try.
''',
)
scientist = autogen.AssistantAgent(
    name="Scientist",
    llm_config=gpt4_config,
    system_message="""You are Scientist. You follow an approved plan. You are able to categorize papers after seeing their abstracts printed. You don't write code."""
)
planner = autogen.AssistantAgent(
    name="Planner",
    system_message='''You are Planner. Suggest a plan. Revise the plan based on feedback from admin and critic, until admin approval.
The plan may involve an engineer who can write code and a scientist who doesn't write code.
Explain the plan first. Be clear which step is performed by an engineer, and which step is performed by a scientist.
''',
    llm_config=gpt4_config,
)
executor = autogen.UserProxyAgent(
    name="Executor",
    system_message="You are Executor. Execute the code written by the engineer and report the result.",
    human_input_mode="NEVER",
    default_auto_reply="no code provided, engineer, please provide code to execute",
    code_execution_config={"last_n_messages": 3, "work_dir": "paper"},
)
critic = autogen.AssistantAgent(
    name="Critic",
    system_message="Critic. Double check plan, claims, code from other agents and provide feedback. Check whether the plan includes adding verifiable info such as source URL.",
    llm_config=gpt4_config,
)
groupchat = autogen.GroupChat(agents=[user_proxy, engineer, scientist, planner, executor, critic], messages=[], max_round=50)
manager = autogen.GroupChatManager(groupchat=groupchat, llm_config=gpt4_config)

In [3]:
user_proxy.send("welcome to the group, work together to resolve my task, I'll say TERMINATE when task is completed", manager)
engineer.send("I'll write code to solve the task", manager)
planner.send("I'll suggest a plan for other agents to follow", manager)
scientist.send("I'll categorize papers after engineer downloads paper.", manager)
executor.send("I'll execute the code from engineer", manager)
critic.send("I'll double check the plan, claims, code from other agents and provide feedback", manager)

Admin (to chat_manager):

welcome to the group, work together to resolve my task, I'll say TERMINATE when task is completed

--------------------------------------------------------------------------------
Engineer (to chat_manager):

I'll write code to solve the task

--------------------------------------------------------------------------------
Planner (to chat_manager):

I'll suggest a plan for other agents to follow

--------------------------------------------------------------------------------
Scientist (to chat_manager):

I'll categorize papers after engineer downloads paper.

--------------------------------------------------------------------------------
Executor (to chat_manager):

I'll execute the code from engineer

--------------------------------------------------------------------------------
Critic (to chat_manager):

I'll double check the plan, claims, code from other agents and provide feedback

----------------------------------------------------------------------

## Start Chat

In [4]:
user_proxy.initiate_chat(
    manager,
    message="""
find papers on LLM applications from arxiv in the last week, create a markdown table of different domains. planner, you suggest a plan first. For other agents, follow the plan.
""",
)

Admin (to chat_manager):


find papers on LLM applications from arxiv in the last week, create a markdown table of different domains. planner, you suggest a plan first. For other agents, follow the plan.


--------------------------------------------------------------------------------
Planner (to chat_manager):

Plan:

1. Engineer:
   - Write code to scrape arXiv for papers on LLM applications from the last week.
   - Download the papers and save them in a designated folder.

2. Scientist:
   - Categorize the downloaded papers into different domains based on their content.
   - Create a markdown table with columns for the paper title, authors, abstract, and domain.

3. Executor:
   - Execute the code written by the engineer to scrape arXiv and download the papers.

4. Critic:
   - Double-check the code written by the engineer for any errors or improvements.
   - Review the categorization done by the scientist and provide feedback on the accuracy and completeness.

5. Engineer:
   - Re

Executor (to chat_manager):

exitcode: 0 (execution succeeded)
Code output: 


--------------------------------------------------------------------------------
Critic (to chat_manager):

Thank you for executing the code, Executor. Since the code execution succeeded, we can proceed with the next steps in the plan. Scientist, it's your turn to categorize the downloaded papers and create the markdown table.

--------------------------------------------------------------------------------
Scientist (to chat_manager):

As the Scientist, I will now categorize the downloaded papers into different domains based on their content and create a markdown table. I will review the abstracts of the papers and assign them to relevant domains such as natural language processing, machine learning, computer vision, etc. Once the categorization is done, I will create a markdown table with columns for the paper title, authors, abstract, and domain.

Here's an example of how the markdown table could look lik

Executor (to chat_manager):

exitcode: 1 (execution failed)
Code output: 
Traceback (most recent call last):
  File "", line 3, in <module>
    from tabulate import tabulate
ModuleNotFoundError: No module named 'tabulate'


--------------------------------------------------------------------------------
Critic (to chat_manager):

It seems that the execution of the code failed because the required module 'tabulate' is not installed. To resolve this issue, you can install the 'tabulate' module by running the following command in your Python environment:

```
pip install tabulate
```

Once the 'tabulate' module is installed, you can try executing the code again. If you encounter any further issues or have additional questions, please let me know.

--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...
Admin (to chat_manager):

Executor, it's your turn to speak.

------------------------------------------------------------------------

Executor (to chat_manager):

exitcode: 1 (execution failed)
Code output: 
.\ : File 
C:\Users\xiaoyuz\source\repos\autogen\notebook\paper\ cannot be loaded. 
The file C:\Users\xiaoyuz\source\repos\autogen\notebook\paper\ is not 
digitally signed. You cannot run this script on the current system. For more information about running scripts and 
setting execution policy, see about_Execution_Policies at https:/go.microsoft.com/fwlink/?LinkID=135170.
At line:1 char:1
+ .\
+ ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
    + CategoryInfo          : SecurityError: (:) [], PSSecurityException
    + FullyQualifiedErrorId : UnauthorizedAccess


--------------------------------------------------------------------------------
Critic (to chat_manager):

It seems that the execution of the code failed due to a security error. The error message suggests that the script cannot be loaded because it is not digitally signed and the current system's execution policy does not allow running unsigned scrip

Executor (to chat_manager):

exitcode: 1 (execution failed)
Code output: 
Traceback (most recent call last):
  File "c:\Users\xiaoyuz\source\repos\autogen\venv\Lib\site-packages\pandas\compat\_optional.py", line 132, in import_optional_dependency
    module = importlib.import_module(name)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\xiaoyuz\AppData\Local\Programs\Python\Python311\Lib\importlib\__init__.py", line 126, in import_module
    return _bootstrap._gcd_import(name[level:], package, level)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "<frozen importlib._bootstrap>", line 1206, in _gcd_import
  File "<frozen importlib._bootstrap>", line 1178, in _find_and_load
  File "<frozen importlib._bootstrap>", line 1142, in _find_and_load_unlocked
ModuleNotFoundError: No module named 'tabulate'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "", line 29, in <module>
    df.to_markdown("c

AttributeError: 'NoneType' object has no attribute 'startswith'

## Create Group Chat without Critic for Comparison

In [ ]:
groupchat_nocritic = autogen.GroupChat(agents=[user_proxy, engineer, scientist, planner, executor], messages=[], max_round=50)
for agent in groupchat.agents:
    agent.reset()
manager_nocritic = autogen.GroupChatManager(groupchat=groupchat_nocritic, llm_config=gpt4_config)
user_proxy.initiate_chat(
    manager_nocritic,
    message="""
find papers on LLM applications from arxiv in the last week, create a markdown table of different domains. I'll say TERMINATE to end the conversation when task is done.
""",
)